In [1]:
import pandas as pd
import numpy as np
import random
import pickle
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
from sklearn.ensemble import ExtraTreesRegressor

In [2]:
pkl_file = open('data_qt_norm.pkl', 'rb')
data_transformed = pickle.load(pkl_file)
pkl_file.close()

In [3]:
Y_train = data_transformed['Y_train']
X_train = data_transformed['X_train']
Y_test = data_transformed['Y_test']
X_test = data_transformed['X_test']

In [4]:
Corr_=[]
for col in X_train.columns.values:
    Corr_.append([col, abs(Y_train.corr(X_train[col]))])

In [5]:
Corr_df = pd.DataFrame(Corr_,columns=['column', 'correlation'])
Corr_df.sort_values('correlation',ascending=False).head(10)

,column,correlation
1,timestamp,0.007916
87,technical_20,0.007900
61,fundamental_55,0.007423
17,fundamental_11,0.006672
95,technical_30,0.006339
9,fundamental_2,0.006084
62,fundamental_56,0.006014
59,fundamental_53,0.005658
54,fundamental_48,0.005559
105,technical_40,0.005519


In [6]:
KBest_select = SelectKBest(score_func =f_regression, k=10).fit(X_train, Y_train)

In [7]:
names = X_train.columns.values
kbest_scores=pd.DataFrame({'feature': names, 'score': KBest_select.scores_}).sort_values('score',ascending=False)
kbest_scores.head(10)

,feature,score
1,timestamp,80.407068
87,technical_20,80.087074
61,fundamental_55,70.695912
17,fundamental_11,57.123460
95,technical_30,51.559980
9,fundamental_2,47.492485
62,fundamental_56,46.400596
59,fundamental_53,41.072149
54,fundamental_48,39.645995
105,technical_40,39.079266


In [8]:
Extrees_select = ExtraTreesRegressor(n_estimators=10,random_state=0, n_jobs=8).fit(X_train, Y_train)

In [9]:
Extrees_scores=pd.DataFrame({'feature': names, 'score': Extrees_select.feature_importances_}).sort_values('score',ascending=False)
Extrees_scores.head(10)

,feature,score
98,technical_33,0.047540
106,technical_41,0.041935
95,technical_30,0.039232
90,technical_24,0.037954
73,technical_3,0.035498
71,technical_1,0.033272
74,technical_5,0.031601
96,technical_31,0.029540
87,technical_20,0.029005
91,technical_25,0.028753


In [10]:
selected_corr= list(Corr_df.sort_values('correlation',ascending=False).column)[0:10]
selected_kbest=list(kbest_scores.feature)[0:10]
selected_trees=list(Extrees_scores.feature)[0:10]

In [11]:
selected_features={'selected_corr':selected_corr,'selected_kbest':selected_kbest, 'selected_trees':selected_trees}
output = open('selected_features_qt_norm.pkl', 'wb')
pickle.dump(selected_features, output)
output.close()